In [223]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama
import pandas as pd
import json

## 提取出文本中的物品

In [224]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOllama(temperature=0, model="qwen:32b-chat-v1.5-q5_K_M")

text_input = """
林克推开门进入卧室，房间布置简洁而温馨。一张坚固的木床靠在北墙正中，床头上方挂着一幅描绘着塞尔达大陆的装饰画，床尾朝向南边的窗户，让林克在清晨醒来时能第一时间感受到阳光的温暖。
床的右侧是一个小巧的书桌，书桌面向东边，林克可以在这里规划他的冒险路线，书桌上整齐地摆放着地图和羽毛笔。在房间的西南角，一个高挑的衣柜紧贴墙壁，柜门上刻有海拉尔的纹章，内部整齐地挂着林克的服装和装备。
衣柜对面，一张舒适的靠背椅放置在东墙边，旁边是一张小茶几，上面常放着一杯热茶，供林克在休息时享用。整个房间的布局以方便和实用为主，同时不失为一个放松和规划冒险的理想空间。
"""

get_object_chain = ChatPromptTemplate.from_template("请提取出如下文本中的陈设和物品，输出一个数组。文本：{text_input}") | chat_model | StrOutputParser()

objects_in_text = get_object_chain.invoke({"text_input": text_input})

print(objects_in_text)

['木床', '装饰画', '书桌', '地图', '羽毛笔', '衣柜', '海拉尔的纹章', '服装', '装备', '靠背椅', '小茶几', '热茶']


## 在素材库中找到匹配的物品

In [226]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_in_gallery", description="这是你觉得与用户输入最匹配的物品"),
    ResponseSchema(name="objects_url_in_gallery", description="这是这个最匹配的物品的URL"),
    ResponseSchema(name="objects_x_size", description="这是这个最匹配的物品的X轴尺寸"),
    ResponseSchema(name="objects_y_size", description="这是这个最匹配的物品的Y轴尺寸"),        
    ResponseSchema(name="objects_z_size", description="这是这个最匹配的物品的Z轴尺寸"),           
    ResponseSchema(name="match_score", description="你认为用户输入的匹配与你匹配的接近程度的0-100分")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。
已入库物品的URL和已入库物品是一一对应关系。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

已入库物品:
{objects_in_gallery}

已入库物品URL:
{objects_url_in_gallery}

已入库物品X轴尺寸:
{objects_x_size}

已入库物品Y轴尺寸:
{objects_y_size}

已入库物品Z轴尺寸:
{objects_z_size}


你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "objects_in_gallery"],
    partial_variables={"format_instructions": format_instructions}
)

# Get your standardized names. You can swap this out with whatever list you want!
df = pd.read_csv('../data/ObjectsInGallery.csv')
objects_in_gallery = ", ".join(df['物品'].values)
objects_url_in_gallery = ", ".join(df['URL'].values)
objects_x_size = df['X轴尺寸'].values
objects_y_size = df['Y轴尺寸'].values
objects_z_size = df['Z轴尺寸'].values

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, objects_in_gallery=objects_in_gallery, objects_url_in_gallery=objects_url_in_gallery, 
                              objects_x_size=objects_x_size, objects_y_size=objects_y_size, objects_z_size=objects_z_size)

print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

# 提取JSON内容
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsMatched.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"objects_url_in_gallery": string  // 这是这个最匹配的物品的URL
	"objects_x_size": string  // 这是这个最匹配的物品的X轴尺寸
	"objects_y_size": string  // 这是这个最匹配的物品的Y轴尺寸
	"objects_z_size": string  // 这是这个最匹配的物品的Z轴尺寸
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。
已入库物品的URL和已入库物品是一一对应关系。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"objects_url_in_gallery": string  // 这是这个最匹配的物品的URL
	"objects_x_size":

,object_in_text,object_in_gallery,objects_url_in_gallery,objects_x_size,objects_y_size,objects_z_size,match_score
0,木床,床1,D:/Home/Downloads/Furniture/Bed_01/Bed_01.usd,200,100,150,90
1,装饰画,无匹配项,,,,,0
2,书桌,书桌1,D:/Home/Downloads/Furniture/Desk_001/Desk_001.usd,120,80,80,90
3,地图,无匹配项,,,,,0
4,羽毛笔,无匹配项,,,,,0
5,衣柜,衣柜1,D:/Home/Downloads/Furniture/Armoire_001/Armoir...,120,200,50,90
6,海拉尔的纹章,无匹配项,,,,,0
7,服装,无匹配项,,,,,0
8,装备,无匹配项,,,,,0
9,靠背椅,靠背椅1,D:/Home/Downloads/Furniture/Chair_001/Chair_00...,60,120,60,90


## 根据文本确定物品方位和坐标

In [248]:
# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

# 根据匹配信息整理输入布局的数据
objects_matched = []
objects_matched_x_size = []
objects_matched_y_size = []
objects_matched_z_size = []
for obj in ObjectsMatched:
    if int(obj['match_score']) > 0:
        objects_matched.append(obj['object_in_text'])
        objects_matched_x_size.append(obj['objects_x_size'])
        objects_matched_x_size.append(obj['objects_y_size'])
        objects_matched_x_size.append(obj['objects_z_size'])

    
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_name", description="这是用户输入的物品的英语翻译（合法的C语言变量名称形式）"),
    ResponseSchema(name="place", description="这是根据上下文物品的位置"),
    ResponseSchema(name="translation_x", description="这是物品的位置X轴坐标，单位是厘米"),
    ResponseSchema(name="translation_y", description="这是物品的位置Y轴坐标，单位是厘米"),
    ResponseSchema(name="translation_z", description="这是物品的位置Z轴坐标，单位是厘米"),
    ResponseSchema(name="direction", description="这是根据上下文物品的朝向（南、东、北、西）"),
    ResponseSchema(name="rotation_y", description="这是朝向角度（南=0、东=90、北=180、西=270）")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称，以及该物品的X、Y、Z轴尺寸
根据上下文推断出物品在场景中的位置和朝向。X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800
重点考虑物品之间的相对位置关系，位置应符合上下文的描述，同时避免重叠和碰撞。
如果有描述边界，物品不要超出边界。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_matched}

物品的X轴尺寸：
{objects_matched_x_size}

物品的Y轴尺寸：
{objects_matched_y_size}

物品的Z轴尺寸：
{objects_matched_z_size}

上下文:
{text_input}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "text_input"],
    partial_variables={"format_instructions": format_instructions}
)

# Your user input
_input = prompt.format_prompt(objects_matched=objects_matched, text_input=text_input, 
                              objects_matched_x_size=objects_matched_x_size, objects_matched_y_size=objects_matched_y_size, objects_matched_z_size=objects_matched_z_size)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsLayout.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式）
	"place": string  // 这是根据上下文物品的位置
	"translation_x": string  // 这是物品的位置X轴坐标，单位是厘米
	"translation_y": string  // 这是物品的位置Y轴坐标，单位是厘米
	"translation_z": string  // 这是物品的位置Z轴坐标，单位是厘米
	"direction": string  // 这是根据上下文物品的朝向（南、东、北、西）
	"rotation_y": string  // 这是朝向角度（南=0、东=90、北=180、西=270）
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称，以及该物品的X、Y、Z轴尺寸
根据上下文推断出物品在场景中的位置和朝向。X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南
重点考虑物品之间的相对位置关系，位置应符合上下文的描述，同时避免重叠和碰撞。
如果有描述边界，物品不要超出边界。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_name": st

,object_in_text,object_name,place,translation_x,translation_y,translation_z,direction,rotation_y
0,木床,wooden_bed,靠北墙正中,0,0,-150,南,180
1,书桌,desk,床的右侧，面向东边,200,0,-150,东,90
2,衣柜,wardrobe,西南角，紧贴墙壁,-120,-50,-150,南,180
3,靠背椅,armchair,东墙边，衣柜对面,200,-50,-150,南,180


## 将JSON文件转换为USDA文件

In [250]:
import json
import os
from pxr import Usd, UsdGeom, Sdf, Gf

# 定义一个函数来创建USD对象
def create_usd_object(stage, prim_path, obj_name, obj_url, position, rotation):
    # 创建一个新的Prim在给定的路径上
    prim = stage.DefinePrim(prim_path, 'Xform')
    prim.name = obj_name

    xformable = UsdGeom.Xformable(prim)

    xformable.AddTranslateOp().Set(value=translation)
    xformable.AddRotateXYZOp().Set(value=rotation)

    # 创建payload，并设置其references指向目标USD文件
    payload = Sdf.Payload(obj_url)
    prim.GetPayloads().AddPayload(payload)

    print(xformable)
    
    return prim


# 创建USD stage
stage = Usd.Stage.CreateNew('../data/link_room_layout_33.usda')

# 创建一个新的Prim，这将作为默认Prim
default_prim = stage.DefinePrim('/Objects')

# 使用SetDefaultPrim方法设置默认Prim
stage.SetDefaultPrim(default_prim)

# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

with open('../data/ObjectsLayout.json', 'r') as f:
    ObjectsLayout = json.load(f)

# # 合并两个JSON文件的数据
for obj in ObjectsMatched:
    if int(obj['match_score']) <= 0:
        continue

    for obj_layout in ObjectsLayout:
        if obj_layout['object_in_text'] == obj['object_in_text']:
            obj_layout.update(obj)

for o in ObjectsLayout:
    print(o)

for o in ObjectsMatched:
    print(o)

# 遍历JSON数据，为每个对象创建一个USD prim
for obj in ObjectsLayout:
    if not 'objects_url_in_gallery' in obj:
        continue

    prim_path = '/Objects/' + obj['object_name']
    translation = Gf.Vec3f(float(obj['translation_x']), float(obj['translation_y']), float(obj['translation_z']))
    rotation = Gf.Vec3f(0, float(obj['rotation_y']), 0)
    create_usd_object(stage, prim_path, obj['object_in_text'], obj['objects_url_in_gallery'], translation, rotation)

# 保存USD stage
stage.Save()


{'object_in_text': '木床', 'object_name': 'wooden_bed', 'place': '靠北墙正中', 'translation_x': '0', 'translation_y': '0', 'translation_z': '-150', 'direction': '南', 'rotation_y': '180', 'object_in_gallery': '床1', 'objects_url_in_gallery': 'D:/Home/Downloads/Furniture/Bed_01/Bed_01.usd', 'objects_x_size': '200', 'objects_y_size': '100', 'objects_z_size': '150', 'match_score': '90'}
{'object_in_text': '书桌', 'object_name': 'desk', 'place': '床的右侧，面向东边', 'translation_x': '200', 'translation_y': '0', 'translation_z': '-150', 'direction': '东', 'rotation_y': '90', 'object_in_gallery': '书桌1', 'objects_url_in_gallery': 'D:/Home/Downloads/Furniture/Desk_001/Desk_001.usd', 'objects_x_size': '120', 'objects_y_size': '80', 'objects_z_size': '80', 'match_score': '90'}
{'object_in_text': '衣柜', 'object_name': 'wardrobe', 'place': '西南角，紧贴墙壁', 'translation_x': '-120', 'translation_y': '-50', 'translation_z': '-150', 'direction': '南', 'rotation_y': '180', 'object_in_gallery': '衣柜1', 'objects_url_in_gallery': 'D